In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [2]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/An-Nasr/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/An-Nasr/metadata_110.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,3_1.wav,4,110,An-Nasr
1,3_2.wav,4,110,An-Nasr
2,3_3.wav,4,110,An-Nasr
3,3_4.wav,4,110,An-Nasr
4,2_1.wav,3,110,An-Nasr
5,2_2.wav,3,110,An-Nasr
6,2_3.wav,3,110,An-Nasr
7,2_4.wav,3,110,An-Nasr
8,1_1.wav,1,110,An-Nasr
9,1_2.wav,1,110,An-Nasr


Feature Extraction

In [3]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [4]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [5]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00179377  0.00487049  0.00431925 ... -0.15333343 -0.13051389
 -0.15450318] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.06791052  0.02416206  0.01926574 ...  0.1288552   0.16924351
  0.20366314] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.10735187 -0.06064139 -0.08216329 ... -0.04587401 -0.05375952
 -0.00395003] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.26102832  0.29

In [6]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (4, 704, 40)
Shape of labels for fold 1: (4,)


In [7]:
mfcc_data_by_fold[3]["data"].shape

(4, 704, 40)

In [8]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [9]:
mfcc_data_by_fold[1]["data"].shape

(4, 704, 40, 1)

After Training Work, Deep Features 

In [10]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 58ms/step


In [11]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [12]:
intermediate_layer_outputs.shape

(3, 4, 1024)

In [13]:
intermediate_layer_outputs

array([[[-0.9999135 , -0.9999997 ,  0.99999493, ...,  0.9967931 ,
         -0.9996835 , -0.99990237],
        [-0.99692655,  0.9936453 ,  1.        , ...,  0.99973905,
         -0.9662983 , -0.999993  ],
        [-0.95497555, -0.5871579 ,  0.9993012 , ...,  0.99623895,
         -0.9001138 , -0.99999136],
        [-0.99998343, -0.9959189 ,  0.9999933 , ...,  0.99837726,
         -0.99964947, -1.        ]],

       [[-0.9663701 , -0.9998363 ,  0.99884135, ...,  0.9996569 ,
         -0.9974964 , -0.99913484],
        [-0.88986105,  0.46070212,  0.99761826, ...,  0.99996036,
         -0.9983824 , -0.9999352 ],
        [-0.98454285,  0.4226775 ,  0.9999793 , ...,  0.9998223 ,
         -0.99489975, -0.99998766],
        [-0.95427567, -0.24313056,  0.9993921 , ...,  0.99865544,
         -0.99993366, -0.9999926 ]],

       [[-0.99416983, -0.9991154 ,  0.79551303, ...,  0.9983113 ,
         -0.95728   , -0.9862086 ],
        [-0.9454105 ,  1.        ,  0.7228058 , ...,  0.9984402 ,
         -0.

In [14]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [15]:
an_nasr_mean = mean_across_folds

In [16]:
an_nasr_mean.shape

(4, 1024)

In [17]:
an_nasr_mean

array([[-0.98681784, -0.9996505 ,  0.9314497 , ...,  0.99825376,
        -0.98481995, -0.99508196],
       [-0.94406605,  0.8181158 ,  0.90680796, ...,  0.9993799 ,
        -0.98583317, -0.99914616],
       [-0.9690633 ,  0.01941904,  0.9948481 , ...,  0.99495393,
        -0.8398431 , -0.9997792 ],
       [-0.9798277 , -0.07969788,  0.99428004, ...,  0.99586755,
        -0.9458191 , -0.9939478 ]], dtype=float32)

In [18]:
# Specify the file path where you want to save the .npy file
file_path = 'an_nasr_mean.npy'

# Save the array as a .npy file
np.save(file_path, an_nasr_mean)

Testing of Single Audio

In [19]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/nasr_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00179377  0.00487049  0.00431925 ... -0.15333343 -0.13051389
 -0.15450318] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.06791052  0.02416206  0.01926574 ...  0.1288552   0.16924351
  0.20366314] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.10735187 -0.06064139 -0.08216329 ... -0.04587401 -0.05375952
 -0.00395003] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.26102832  0.29242945  0.26738238

In [20]:
mfcc_extracted_features_single

array([[[[-4.6013333e+02],
         [ 6.5609756e+01],
         [ 2.1376125e+01],
         ...,
         [ 7.6098692e-01],
         [-2.7450769e+00],
         [-1.3067177e+01]],

        [[-4.3865445e+02],
         [ 7.1171974e+01],
         [ 1.6183195e+01],
         ...,
         [-1.1471810e+00],
         [-1.2708418e+01],
         [-1.4497263e+01]],

        [[-3.5120190e+02],
         [ 7.0384903e+01],
         [ 3.1417423e+01],
         ...,
         [ 6.5818250e-01],
         [-1.1579357e+01],
         [-7.5934291e+00]],

        ...,

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
 

In [21]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 124ms/step


In [22]:
intermediate_output

array([[-0.99999756, -1.        ,  0.9999996 , ...,  0.99886787,
        -0.9833616 , -0.99988276]], dtype=float32)

In [23]:
intermediate_output.shape

(1, 1024)

In [24]:
intermediate_layer_outputs

array([[[-0.9999135 , -0.9999997 ,  0.99999493, ...,  0.9967931 ,
         -0.9996835 , -0.99990237],
        [-0.99692655,  0.9936453 ,  1.        , ...,  0.99973905,
         -0.9662983 , -0.999993  ],
        [-0.95497555, -0.5871579 ,  0.9993012 , ...,  0.99623895,
         -0.9001138 , -0.99999136],
        [-0.99998343, -0.9959189 ,  0.9999933 , ...,  0.99837726,
         -0.99964947, -1.        ]],

       [[-0.9663701 , -0.9998363 ,  0.99884135, ...,  0.9996569 ,
         -0.9974964 , -0.99913484],
        [-0.88986105,  0.46070212,  0.99761826, ...,  0.99996036,
         -0.9983824 , -0.9999352 ],
        [-0.98454285,  0.4226775 ,  0.9999793 , ...,  0.9998223 ,
         -0.99489975, -0.99998766],
        [-0.95427567, -0.24313056,  0.9993921 , ...,  0.99865544,
         -0.99993366, -0.9999926 ]],

       [[-0.99416983, -0.9991154 ,  0.79551303, ...,  0.9983113 ,
         -0.95728   , -0.9862086 ],
        [-0.9454105 ,  1.        ,  0.7228058 , ...,  0.9984402 ,
         -0.

Euclidian Distance with Mean

In [25]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(an_nasr_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = an_nasr_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in an_nasr_mean: {closest_index+1}")
print(f"Closest value in an_nasr_mean: {closest_value}")

Index of closest value in an_nasr_mean: 1
Closest value in an_nasr_mean: [-0.98681784 -0.9996505   0.9314497  ...  0.99825376 -0.98481995
 -0.99508196]


Euclidian Distance Without Mean

In [26]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")